## Assignment A - OpenFDA Query to Get AZ Drugs and Avg Ingredients per Year
The assignment is described as follows:

Using the data from the OpenFDA API (documentation at https://open.fda.gov/apis/drug/label/) to determine the average number of ingredients (spl_product_data_elements) contained in AstraZeneca medicines per year.

Starting with having a look at the API and making some calls in the browser. After some initial exploration, + having had a look at the xml file of categories available, I settle on: https://api.fda.gov/drug/label.json?search=openfda.brand_name=AstraZeneca+AND+_exists_:openfda.generic_name&limit=99
- brand_name=Astrazeneca because only interested in AZ drugs
- _exists_:openfda.generic_name because the drug name field must be populated (for achieving second column in assignment description)
- limit=99 because unless specified only 1 hit is returned

count=spl_product_data_elements.exact is not included in the query. I'll explain why further down.

Ready to start. Importing relevant files:

In [1]:
import json
import urllib
import pandas as pd 
from pandas.io.json import json_normalize #for flattening json files -> pandas df

In [2]:
#load json object from url
with urllib.request.urlopen('https://api.fda.gov/drug/label.json?search=openfda.brand_name=AstraZeneca'+\
                            '+AND+_exists_:openfda.generic_name&limit=99') as url:
    data = json.loads(url.read().decode()) #data to python dict

In [3]:
#total nr of hits for the search above
total_hits = data['meta']['results']['total']
total_hits

303

To gather all relevant data, I iterate to get data where there is a drug name and brand name is AZ, which should be 303 hits as seen above. I'll later use the effective_time field to split the data by date.

In [4]:
#need to iterate so that all data is there, the number of times is as follows:
times = total_hits//data['meta']['results']['limit'] + 1 #times divided plus 1 extra to get the rest
data_list = [] #list to hold the dfs from each iteration to concat later
for i in range(times):
    with urllib.request.urlopen('https://api.fda.gov/drug/label.json?search=openfda.brand_name=AstraZeneca'+\
                                '+AND+_exists_:openfda.generic_name&skip='+str(i*99)+'&limit=99') as url:
        search_results = json.loads(url.read().decode())
        data_list.append(json_normalize(search_results['results']))#turn json into df and append to data_list

df = pd.concat(data_list, ignore_index=True, sort=False)
df.info() #quick look to see the numbers are right

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Columns: 109 entries, abuse to precautions_table
dtypes: object(109)
memory usage: 258.1+ KB


Looks like the data is right; 303 rows. Number of columns a bit excessive; might be an issue there but won't prioritize investigation. Now I only need a few columns to be able to achieve the format specified in the assignment + other formatting to  remove drug name duplicates, remove dates (keep year), sort and aggregate data. Assumption: generic_name duplicates are the same drug despite some potential differences (concentration of API/delivery method etc).

In [5]:
#only need effective_date, generic_name and spl_product_data_elements:
df = df[['effective_time', 'openfda.generic_name', 'spl_product_data_elements']] #won't need the other columns; overwrite
df.head()

,effective_time,openfda.generic_name,spl_product_data_elements
0,20180615,[DAPAGLIFLOZIN],[FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDI...
1,20090706,[BICALUTAMIDE],[Bicalutamide Bicalutamide BICALUTAMIDE BICALU...
2,20171101,[BICALUTAMIDE],[CASODEX Bicalutamide BICALUTAMIDE BICALUTAMID...
3,20180609,[CLOPIDOGREL],[clopidogrel clopidogrel CLOPIDOGREL BISULFATE...
4,20181129,[QUETIAPINE FUMARATE],[SEROQUEL Quetiapine fumarate QUETIAPINE FUMAR...


In [6]:
#slice dates: all years are at beginning and 4 chars long
df['effective_time']=df['effective_time'].str.slice(0,4)
df.head()

,effective_time,openfda.generic_name,spl_product_data_elements
0,2018,[DAPAGLIFLOZIN],[FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDI...
1,2009,[BICALUTAMIDE],[Bicalutamide Bicalutamide BICALUTAMIDE BICALU...
2,2017,[BICALUTAMIDE],[CASODEX Bicalutamide BICALUTAMIDE BICALUTAMID...
3,2018,[CLOPIDOGREL],[clopidogrel clopidogrel CLOPIDOGREL BISULFATE...
4,2018,[QUETIAPINE FUMARATE],[SEROQUEL Quetiapine fumarate QUETIAPINE FUMAR...


In [7]:
#removing list format from other columns
df['openfda.generic_name'] = df['openfda.generic_name'].str[0]
df['spl_product_data_elements'] = df['spl_product_data_elements'].str[0]
df.head()

,effective_time,openfda.generic_name,spl_product_data_elements
0,2018,DAPAGLIFLOZIN,FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDIO...
1,2009,BICALUTAMIDE,Bicalutamide Bicalutamide BICALUTAMIDE BICALUT...
2,2017,BICALUTAMIDE,CASODEX Bicalutamide BICALUTAMIDE BICALUTAMIDE...
3,2018,CLOPIDOGREL,clopidogrel clopidogrel CLOPIDOGREL BISULFATE ...
4,2018,QUETIAPINE FUMARATE,SEROQUEL Quetiapine fumarate QUETIAPINE FUMARA...


In [8]:
#remove duplicates in the openfda.generic_name column
df = df.drop_duplicates(['openfda.generic_name'], keep='first')
df.head(10)

,effective_time,openfda.generic_name,spl_product_data_elements
0,2018,DAPAGLIFLOZIN,FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDIO...
1,2009,BICALUTAMIDE,Bicalutamide Bicalutamide BICALUTAMIDE BICALUT...
3,2018,CLOPIDOGREL,clopidogrel clopidogrel CLOPIDOGREL BISULFATE ...
4,2018,QUETIAPINE FUMARATE,SEROQUEL Quetiapine fumarate QUETIAPINE FUMARA...
5,2017,ANASTROZOLE,Anastrozole Anastrozole ANASTROZOLE ANASTROZOL...
6,2011,BUDESONIDE,PULMICORTFLEXHALER Budesonide BUDESONIDE BUDES...
7,2013,CANDESARTAN CILEXETIL AND HYDROCHLOROTHIAZIDE,ATACAND HCT Candesartan cilexetil and Hydrochl...
8,2018,LISINOPRIL AND HYDROCHLOROTHIAZIDE,Zestoretic lisinopril and hydrochlorothiazide ...
9,2018,CLOPIDOGREL BISULFATE,clopidogrel bisulfate clopidogrel bisulfate HY...
11,2018,METOPROLOL SUCCINATE,Metoprolol succinate Metoprolol succinate ACET...


In [9]:
df = df.groupby('effective_time').agg({'openfda.generic_name':lambda x: ', '.join(x), 
                         'spl_product_data_elements':lambda x: ', '.join(x)}).reset_index()
df

,effective_time,openfda.generic_name,spl_product_data_elements
0,2009,"BICALUTAMIDE, OMEPRAZOLE, FELODIPINE",Bicalutamide Bicalutamide BICALUTAMIDE BICALUT...
1,2010,"ZAFIRLUKAST, LIDOCAINE HYDROCHLORIDE",ACCOLATE Zafirlukast ZAFIRLUKAST ZAFIRLUKAST C...
2,2011,BUDESONIDE,PULMICORTFLEXHALER Budesonide BUDESONIDE BUDES...
3,2013,"CANDESARTAN CILEXETIL AND HYDROCHLOROTHIAZIDE,...",ATACAND HCT Candesartan cilexetil and Hydrochl...
4,2014,ROSUVASTATIN CALCIUM,CRESTOR Rosuvastatin calcium ROSUVASTATIN CALC...
5,2015,TAMOXIFEN CITRATE,Tamoxifen Citrate Tamoxifen Citrate TAMOXIFEN ...
6,2016,"CANDESARTAN CILEXETIL, PRAMLINTIDE ACETATE, ME...",Candesartan cilexetil Candesartan cilexetil CA...
7,2017,"ANASTROZOLE, CLOPIDOGREL KIT, PRILOCAINE HYDRO...",Anastrozole Anastrozole ANASTROZOLE ANASTROZOL...
8,2018,"DAPAGLIFLOZIN, CLOPIDOGREL, QUETIAPINE FUMARAT...",FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDIO...
9,2019,"CICLESONIDE, FLUCONAZOLE, EXENATIDE, DAPAGLIFL...",Omnaris ciclesonide CICLESONIDE CICLESONIDE MI...


In the coding exercise, the format is specified to start with 2018 which I assume means 2019 shouldn't be included and that the df should be sorted newest to oldest. Easiest way to remove 2019 from the df above is by its index, then sort the remaining df.

In [10]:
df = df.drop(df.index[9]) #dropping 2019 by its index 9

In [11]:
df['effective_time'] = df['effective_time'].apply(pd.to_numeric) #need years to be numeric for sorting
df.dtypes #checking effective_time was transformed to int

effective_time                int64
openfda.generic_name         object
spl_product_data_elements    object
dtype: object

In [12]:
df = df.sort_values(by=['effective_time'],ascending=False).reset_index() #sorting by effective_time, descending
df

,index,effective_time,openfda.generic_name,spl_product_data_elements
0,8,2018,"DAPAGLIFLOZIN, CLOPIDOGREL, QUETIAPINE FUMARAT...",FARXIGA DAPAGLIFLOZIN DAPAGLIFLOZIN PROPANEDIO...
1,7,2017,"ANASTROZOLE, CLOPIDOGREL KIT, PRILOCAINE HYDRO...",Anastrozole Anastrozole ANASTROZOLE ANASTROZOL...
2,6,2016,"CANDESARTAN CILEXETIL, PRAMLINTIDE ACETATE, ME...",Candesartan cilexetil Candesartan cilexetil CA...
3,5,2015,TAMOXIFEN CITRATE,Tamoxifen Citrate Tamoxifen Citrate TAMOXIFEN ...
4,4,2014,ROSUVASTATIN CALCIUM,CRESTOR Rosuvastatin calcium ROSUVASTATIN CALC...
5,3,2013,"CANDESARTAN CILEXETIL AND HYDROCHLOROTHIAZIDE,...",ATACAND HCT Candesartan cilexetil and Hydrochl...
6,2,2011,BUDESONIDE,PULMICORTFLEXHALER Budesonide BUDESONIDE BUDES...
7,1,2010,"ZAFIRLUKAST, LIDOCAINE HYDROCHLORIDE",ACCOLATE Zafirlukast ZAFIRLUKAST ZAFIRLUKAST C...
8,0,2009,"BICALUTAMIDE, OMEPRAZOLE, FELODIPINE",Bicalutamide Bicalutamide BICALUTAMIDE BICALUT...


Now comes the tricky part. Original plan was to use the api itself to count exact drugnames in spl_product_data_elements and then average them per year. I managed to get https://api.fda.gov/drug/label.json?search=effective_time:[20180101+TO+20181231]+AND+openfda.brand_name=AstraZeneca+AND+_exists_:openfda.generic_name&count=spl_product_data_elements to work, but it's useless since it counts every word in the drugname as a separate drug.
For some reason, https://api.fda.gov/drug/label.json?search=effective_time:[20180101+TO+20181231]+AND+openfda.brand_name=AstraZeneca+AND+_exists_:openfda.generic_name&count=spl_product_data_elements.exact doesn't yeald any results.

I'll have to find some other logic in spl_product_data_elements to count the average number of ingredients. Quick look at some of the values in spl_product_data_elements:

In [13]:
df['spl_product_data_elements'][5]

'ATACAND HCT Candesartan cilexetil and Hydrochlorothiazide CANDESARTAN CILEXETIL CANDESARTAN HYDROCHLOROTHIAZIDE HYDROCHLOROTHIAZIDE CARBOXYMETHYLCELLULOSE CALCIUM HYDROXYPROPYL CELLULOSE LACTOSE MONOHYDRATE MAGNESIUM STEARATE STARCH, CORN POLYETHYLENE GLYCOL 8000 FERRIC OXIDE YELLOW biconvex ACJ;322, Cilostazol Cilostazol CILOSTAZOL CILOSTAZOL SILICON DIOXIDE CROSPOVIDONE MAGNESIUM STEARATE CELLULOSE, MICROCRYSTALLINE POVIDONES STARCH, CORN white to off-white TEVA;7231'

In [14]:
df['spl_product_data_elements'][2]

'Candesartan cilexetil Candesartan cilexetil CANDESARTAN CILEXETIL CANDESARTAN POLYETHYLENE GLYCOL, UNSPECIFIED HYDROXYPROPYL CELLULOSE (1600000 WAMW) LACTOSE MONOHYDRATE STARCH, CORN CARBOXYMETHYLCELLULOSE CALCIUM MAGNESIUM STEARATE FERRIC OXIDE RED white to off-white circular biconvex shaped ACF;004 Candesartan cilexetil Candesartan cilexetil CANDESARTAN CILEXETIL CANDESARTAN POLYETHYLENE GLYCOL, UNSPECIFIED HYDROXYPROPYL CELLULOSE (1600000 WAMW) LACTOSE MONOHYDRATE STARCH, CORN CARBOXYMETHYLCELLULOSE CALCIUM MAGNESIUM STEARATE FERRIC OXIDE RED light pink circular biconvex-sharped ACG;008 Candesartan cilexetil Candesartan cilexetil CANDESARTAN CILEXETIL CANDESARTAN POLYETHYLENE GLYCOL, UNSPECIFIED HYDROXYPROPYL CELLULOSE (1600000 WAMW) LACTOSE MONOHYDRATE STARCH, CORN CARBOXYMETHYLCELLULOSE CALCIUM MAGNESIUM STEARATE FERRIC OXIDE RED circular biconvex shaped ACH;016 Candesartan cilexetil Candesartan cilexetil CANDESARTAN CILEXETIL CANDESARTAN POLYETHYLENE GLYCOL, UNSPECIFIED HYDROXYP

It seems drug names are separated by commas (which goes nicely with the commas I joined the different strings with). Separating by commas will also help "ignoring" the duplicates that seem to be everywhere (Candesartan cilexetil Candesartan cilexetil CANDESARTAN CILEXETIL).

There are problems with this approach; I see for instance that this string probably contains more than one drug:"Omeprazole magnesium OMEPRAZOLE MAGNESIUM OMEPRAZOLE HYDROXYPROPYL CELLULOSE (TYPE H) HYPROMELLOSES MAGNESIUM STEARATE TALC TRIETHYL CITRATE ANHYDROUS CITRIC ACID XANTHAN GUM ANHYDROUS DEXTROSE GLYCERYL MONOSTEARATE METHACRYLIC ACID POLYSORBATE 80 SODIUM HYDROXIDE ALCOHOL", which means my method is a quite inexact approximation. Not a perfect solution but it'll do. 

In [15]:
#iterating through spl_product_data_elements, also have to count drugs per year to get the average
avg = []
for i in range(len(df['spl_product_data_elements'])):
    ingredients = df['spl_product_data_elements'][i].count(',') + 1 #+1 because items are n+1 if commas separating items are n
    drugs = df['openfda.generic_name'][i].count(',') + 1
    avg.append(round(ingredients/drugs)) #choosing to round to closest int
    
avg #quick look at results

[4, 2, 4, 5, 2, 2, 1, 1, 2]

The average number of ingredients seems way off. Could be improved by additional rules, for instance searching for '('; every time it appears the type/concentration of a certain drug is specified, presumably at the end of its name. Example: UNSPECIFIED HYDROXYPROPYL CELLULOSE (1600000 WAMW). Dots '.' also seem to appear when a certain type (or multiple types) of drug is listed. Example: SODIUM LAURYL SULFATE GELATIN FD&C BLUE NO. 1 FD&C BLUE NO. 2 D&C RED NO. 28. I'll add these rules to improve the result a little.

In [16]:
avg = []
for i in range(len(df['spl_product_data_elements'])):
    ingredients = df['spl_product_data_elements'][i].count(',') +\
    df['spl_product_data_elements'][i].count('(') +\
    df['spl_product_data_elements'][i].count('.') + 1
    drugs = df['openfda.generic_name'][i].count(',') + 1
    avg.append(round(ingredients/drugs))
    
avg #quick look at results

[6, 4, 8, 5, 2, 2, 1, 1, 4]

Slightly better but probably still far from the real number.

New approach: I assembled a list of the possible endings of the most common chemical names. These endings should only appear once per ingredient since chemicals are named based on a pattern. These were the endings I came up with:
	- ol
	- ole
	- ine
	- one
	- ide
	- ene
	- ate
	- acid
    - phene
    - ose
The list probably doesn't cover it all but is still going to be a better approximation.


In [17]:
avg = []
for i in range(len(df['spl_product_data_elements'])):
     #.upper() to handle case-sensitivity of count()
    ingredients = df['spl_product_data_elements'][i].upper().count('OL ') +\
    df['spl_product_data_elements'][i].upper().count('OLE ') +\
    df['spl_product_data_elements'][i].upper().count('INE ') +\
    df['spl_product_data_elements'][i].upper().count('ONE ') +\
    df['spl_product_data_elements'][i].upper().count('IDE ') +\
    df['spl_product_data_elements'][i].upper().count('ENE ') +\
    df['spl_product_data_elements'][i].upper().count('ATE ') +\
    df['spl_product_data_elements'][i].upper().count('ACID ') +\
    df['spl_product_data_elements'][i].upper().count('PHENE ') +\
    df['spl_product_data_elements'][i].upper().count('OSE ')
    drugs = df['openfda.generic_name'][i].count(',') + 1
    avg.append(round(ingredients/drugs))
    
avg #quick look at results

[20, 11, 21, 16, 5, 10, 3, 8, 18]

In [18]:
final_results = pd.DataFrame({
    'year': df['effective_time'],
    'drug_names': df['openfda.generic_name'],
    'avg_number_of_ingredients': avg
})
final_results

,year,drug_names,avg_number_of_ingredients
0,2018,"DAPAGLIFLOZIN, CLOPIDOGREL, QUETIAPINE FUMARAT...",20
1,2017,"ANASTROZOLE, CLOPIDOGREL KIT, PRILOCAINE HYDRO...",11
2,2016,"CANDESARTAN CILEXETIL, PRAMLINTIDE ACETATE, ME...",21
3,2015,TAMOXIFEN CITRATE,16
4,2014,ROSUVASTATIN CALCIUM,5
5,2013,"CANDESARTAN CILEXETIL AND HYDROCHLOROTHIAZIDE,...",10
6,2011,BUDESONIDE,3
7,2010,"ZAFIRLUKAST, LIDOCAINE HYDROCHLORIDE",8
8,2009,"BICALUTAMIDE, OMEPRAZOLE, FELODIPINE",18


My final answer to Assignment A is displayed above. 

## Assignment B - Same as Above, Split by Delivery Route
The time is up by now, but I'd do the following query: 
https://api.fda.gov/drug/label.json?search=openfda.brand_name=AstraZeneca+AND+_exists_:openfda.generic_name+AND+_exists_:openfda.route&limit=99
Then split by route.
